In [30]:
import mobile_traffic as mt
from datetime import time
import noise
import numpy as np

In [31]:
from Sleep.SessionDistribution import calculate_session_distribution_city

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
path = '/Users/anmusso/Desktop/PhD/Projects/Current/NetMob/NetMobCode/temp'

# Expected Time of Session Data

In [66]:
mobile_traffic_lyon = mt.get_city_traffic_data(traffic_type=mt.TrafficType.USERS, city=mt.City.LYON, geo_data_type=mt.GeoDataType.IRIS)

100%|██████████| 77/77 [00:30<00:00,  2.50it/s]


In [67]:
session_distribution_lyon = calculate_session_distribution_city(mobile_traffic_lyon, start=time(21), end=time(3,30), window_smoothing=3)

In [68]:
expected_session_time = session_distribution_lyon.expected_time_of_session_by_location()
expected_session_time.reset_index(names='tile', inplace=True)

In [69]:
expected_session_time.head()

,tile,expected_time
0,011050000,23:00:00
1,012380000,23:00:00
2,012480000,23:00:00
3,012490101,23:00:00
4,012490103,23:00:00


In [70]:
expected_session_time.to_csv(f'{path}/expected_session_time_paris.csv', index=False)

# Noise Data

In [61]:
paris_geo = mt.geo_tile.get_geo_data(city=mt.City.PARIS)
paris_geo = paris_geo.iloc[:100]

In [62]:
noise_estimates = noise.get_noise_estimate(polygons=paris_geo, city=noise.City.PARIS, measurement=noise.Measurement.NIGHT)

In [63]:
noise_estimates = noise_estimates['noise_estimate'].to_frame()
noise_estimates.reset_index(names='tile', inplace=True)

In [64]:
noise_estimates.head()

,tile,noise_estimate
0,162,61.0
1,507,49.0
2,508,61.0
3,853,52.0
4,854,60.0


In [65]:
noise_estimates.to_csv(f'{path}/noise_estimates_paris_n.csv', index=False)